# MLP Model with Keras for trump selection
Model to determine trump selection.
Generate model is used in determination_monte_carlo_tree_search

### Load and initialize Data

In [7]:
import pandas as pd
from pathlib import Path

path_to_data = Path('../ml/')
# Import only a fraction of data for efficient testing
data = pd.read_csv(path_to_data / '2018_10_18_trump.csv', header=None)
cards = [
    # Diamonds
    'DA', 'DK', 'DQ', 'DJ', 'D10', 'D9', 'D8', 'D7', 'D6',
    # Hearts
    'HA', 'HK', 'HQ', 'HJ', 'H10', 'H9', 'H8', 'H7', 'H6',
    # Spades
    'SA', 'SK', 'SQ', 'SJ', 'S10', 'S9', 'S8', 'S7', 'S6',
    # Clubs
    'CA', 'CK', 'CQ', 'CJ', 'C10', 'C9', 'C8', 'C7', 'C6'
]

# Forehand (yes = 1, no = 0)
forehand = ['FH']

user = ['user']
trump = ['trump']

data.columns = cards + forehand + user + trump
feature_columns = cards + forehand
data.drop('user', axis='columns', inplace=True)
print(data.head())

   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  CK  CQ  CJ  C10  C9  C8  C7  \
0   0   0   0   1    1   0   1   1   0   0  ...   0   1   0    0   0   1   0   
1   0   0   0   0    0   0   0   0   1   1  ...   0   0   1    0   0   0   1   
2   1   0   0   1    0   0   0   0   0   0  ...   0   1   0    0   0   0   1   
3   0   0   0   0    0   0   0   0   0   1  ...   0   0   0    1   1   0   0   
4   0   1   0   0    0   0   0   0   1   1  ...   0   0   1    0   0   0   0   

   C6  FH  trump  
0   0   0      6  
1   0   0      5  
2   1   0      6  
3   0   0      5  
4   0   1      4  

[5 rows x 38 columns]


### Arrange Dataset

In [8]:
dummy1 = pd.get_dummies(data['trump'], drop_first=True)
dummy1.head()
trumps = ['D', 'H', 'S', 'C', 'O', 'U']
dummy1.columns = trumps
dummy1.head()
data = pd.concat([data, dummy1], axis=1).drop('trump', axis=1)
print(data.head())

   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  C8  C7  C6  FH  D  H  S  C  \
0   0   0   0   1    1   0   1   1   0   0  ...   1   0   0   0  0  0  0  0   
1   0   0   0   0    0   0   0   0   1   1  ...   0   1   0   0  0  0  0  0   
2   1   0   0   1    0   0   0   0   0   0  ...   0   1   1   0  0  0  0  0   
3   0   0   0   0    0   0   0   0   0   1  ...   0   0   0   0  0  0  0  0   
4   0   1   0   0    0   0   0   0   1   1  ...   0   0   0   1  0  0  0  1   

   O  U  
0  0  1  
1  1  0  
2  0  1  
3  1  0  
4  0  0  

[5 rows x 43 columns]


### Feature Engineering

In [9]:
for color in 'DHSC':
    # Jack and nine combination
    new_col = '{}_J9'.format(color)
    data[new_col] = data['{}J'.format(color)] & data['{}9'.format(color)]
    feature_columns.append(new_col)

    new_col = '{}_AKQ'.format(color)
    data[new_col] = data['{}A'.format(color)] & data['{}K'.format(color)] & data['{}Q'.format(color)]
    feature_columns.append(new_col)

    new_col = '{}_678'.format(color)
    data[new_col] = data['{}6'.format(color)] & data['{}7'.format(color)] & data['{}8'.format(color)]
    feature_columns.append(new_col)

print(data.head())

   DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  D_678  H_J9  H_AKQ  H_678  \
0   0   0   0   1    1   0   1   1   0   0  ...      0     0      0      0   
1   0   0   0   0    0   0   0   0   1   1  ...      0     0      0      0   
2   1   0   0   1    0   0   0   0   0   0  ...      0     0      0      0   
3   0   0   0   0    0   0   0   0   0   1  ...      0     0      0      0   
4   0   1   0   0    0   0   0   0   1   1  ...      0     0      1      0   

   S_J9  S_AKQ  S_678  C_J9  C_AKQ  C_678  
0     0      0      0     0      0      0  
1     0      0      0     0      0      0  
2     0      0      0     0      0      0  
3     0      0      0     0      0      0  
4     0      0      0     0      0      0  

[5 rows x 55 columns]


### Split data into train & test

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data[trumps], test_size=0.2,
                                                    stratify=data[trumps], random_state=42, shuffle=True)
print(X_train.shape)
print(X_train.head())

(287860, 49)
        DA  DK  DQ  DJ  D10  D9  D8  D7  D6  HA  ...  D_678  H_J9  H_AKQ  \
311259   1   0   0   0    1   0   0   0   0   1  ...      0     0      0   
79927    0   0   1   0    0   1   0   1   1   0  ...      0     0      0   
133742   1   0   1   1    0   0   0   0   0   0  ...      0     0      0   
338250   0   1   0   0    0   1   1   0   0   0  ...      0     0      0   
65075    0   0   0   0    0   1   0   0   0   0  ...      0     0      0   

        H_678  S_J9  S_AKQ  S_678  C_J9  C_AKQ  C_678  
311259      0     0      0      0     0      0      0  
79927       0     0      0      0     0      0      0  
133742      0     0      0      0     1      0      0  
338250      0     0      0      0     0      0      0  
65075       0     0      0      0     1      0      0  

[5 rows x 49 columns]


### Build Keras Model

In [11]:
from sklearn.metrics import accuracy_score
from tensorflow import keras
import matplotlib.pyplot as plt

model = keras.Sequential()
dataset_dim = X_train.shape[1]

model.add(keras.layers.Dense(49, activation=keras.activations.tanh, input_shape=(dataset_dim,)))
model.add(keras.layers.Dense(300, activation=keras.activations.tanh))
model.add(keras.layers.Dense(200, activation=keras.activations.tanh))
# model.add(keras.layers.Dense(200, activation=keras.activations.sigmo id))
# model.add(keras.layers.Dense(100, activation=keras.activations.sigmoid))
model.add(keras.layers.Dense(6, activation=keras.activations.softmax))
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128)


def plot_costs(costs):
    fig, ax = plt.subplots()
    ax.plot(costs)
    ax.set_title("Loss curve")
    plt.show()


plot_costs(history.history["loss"])
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

print("Accuracy: %.4f" % accuracy)

Epoch 1/10
 572/2249 [======>.......................] - ETA: 4s - loss: 0.8016 - accuracy: 0.6106

KeyboardInterrupt: 

### Test Model (Only do onces!!!)

In [ ]:
y_pred = model.predict(X_test)
result = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result)
print(y_pred)

### Save model for later use

In [ ]:
model.save("mlp_model_v2")